# Capstone Project Notebook

This notebook will be used for the Capstone Project course.

---

In [257]:
!pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [258]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# Part 1

### Scraping the Neighbourhood data from wikipedia

In [259]:
wiki = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1012118802") 

soup = BeautifulSoup(wiki.content,'lxml') 

### Reading the data from the table into a dataframe

In [260]:
table = soup.find_all('table')[0]  

data = pd.read_html(str(table)) 

neighbourhood=pd.DataFrame(data[0])

### Dropping all rows where the Borough is 'Not assigned'

In [261]:
neighbourhood = neighbourhood[neighbourhood['Borough'] != 'Not assigned']

### Replacing rows where Neighbourhood is 'Not assigned' with the name of the Borough

In [262]:
neighbourhood.loc[neighbourhood["Neighbourhood"]=='Not assigned', "Neighbourhood"] = neighbourhood["Borough"]

In [263]:
neighbourhood.shape

(103, 3)

In [264]:
neighbourhood.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Part 2

### Importing the Geospatial_Coordinates csv file

In [265]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_c8133f94b0404133a96a1b965250e43a = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_c8133f94b0404133a96a1b965250e43a = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

client_c8133f94b0404133a96a1b965250e43a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='AF9i48uzMISpDC7UAA5SzU39NOL-BuSCy-7l2v2L9iRl',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_c8133f94b0404133a96a1b965250e43a)

body = client_c8133f94b0404133a96a1b965250e43a.get_object(Bucket='dataanalysiswithpython-donotdelete-pr-fng9ingtkouq7b',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

coordinates = pd.read_csv(body)

### Merging the Neighbourhood dataframe with the coordinates dataframe

In [266]:
geo_neigh = pd.merge(neighbourhood, coordinates, how="left", on=["Postal Code"])

In [267]:
geo_neigh.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Part 3

### Creating a map of North York:

(I chose to focus on the borough of North York)

In [268]:
northyork_data = geo_neigh[geo_neigh['Borough'] == 'North York'].reset_index(drop=True)

address = 'North York, Ontario'

geolocator = Nominatim(user_agent="c_p_@live.com")
location = geolocator.geocode(address)
northyork_latitude = location.latitude
northyork_longitude = location.longitude

map_northyork = folium.Map(location=[northyork_latitude, northyork_longitude], zoom_start=11)

for lat, lng, label in zip(northyork_data['Latitude'], northyork_data['Longitude'], northyork_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

### Create a function to get all the venues of each neighbourhood

In [269]:
CLIENT_ID = 'X4XPLSQEZRMXHBPPV35XZNTYQZHSWFIKC4TTANTYS4WZIWXL'
CLIENT_SECRET = 'AKFBTVRQTOWJ20LRD1IRTAADWGCLGFBL31X2T5JY0WR3GPNP'
VERSION = '20180605'
LIMIT = 100 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
northyork_venues = getNearbyVenues(northyork_data["Neighbourhood"], northyork_data["Latitude"], northyork_data["Longitude"])    

### One hot encoding so we can analyze the neighbourhoods

In [ ]:
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")

northyork_onehot['Neighbourhood'] = northyork_venues['Neighbourhood'] 

fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

Group rows by neighbourhood and take the mean of the frequency of occurrence of each category

In [ ]:
northyork_grouped = northyork_onehot.groupby('Neighbourhood').mean().reset_index()

Function to sort the venues in descending order

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a dataframe that shows the top 10 venues per neighbourhood

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhood_venues_sorted = pd.DataFrame(columns=columns)
neighbourhood_venues_sorted['Neighbourhood'] = northyork_grouped['Neighbourhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighbourhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

### Clustering

Run k means with k=6

In [ ]:
kclusters = 6

northyork_grouped_clustering = northyork_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

kmeans.labels_[0:10] 

Create a dataframe that shows the top 10 venues and the cluster for each neighbourhood

In [ ]:
neighbourhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

northyork_merged = northyork_data

northyork_merged = northyork_merged.join(neighbourhood_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

In [ ]:
northyork_merged = northyork_merged.dropna()

### Mapping out the clusters

In [ ]:
map_clusters = folium.Map(location=[northyork_latitude, northyork_longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighbourhood'], northyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [ ]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

### Cluster 2

In [ ]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

### Cluster 3

In [ ]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

### Cluster 4

In [ ]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 3, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

### Cluster 5

In [ ]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 4, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]

### Cluster 6

In [ ]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 5, northyork_merged.columns[[1] + list(range(5, northyork_merged.shape[1]))]]